# Kaggle Cloud Ops: Queen Bee Acoustics + Makueni Apiary Intelligence

This unified notebook stitches together:

1. **Queen Bee acoustic detection (CNN + hyperparameter tuning)**
2. **Makueni Apiary intelligence workflows (weather, NDVI, telemetry, hive stress ML)**

> **Kaggle usage:** Attach the `harshkumar1711/beehive-audio-dataset-with-queen-and-without-queen` dataset plus any `content/main-data` exports as Kaggle data sources. All intermediate files are written under `content/` so the same notebook also works locally.

## BeeUnity System Blueprint

BeeUnity couples two complementary sensing/analytics tracks inside a single reproducible notebook.

- **Acoustic intelligence (Sections §3-15)** ingests the Kaggle beehive audio corpus, generates mel spectrograms, and trains/ tunes a convolutional network for multi-class queen state detection. The outputs are calibrated probabilities + decision thresholds that can be streamed into downstream alerting or fusion models.
- **Makueni apiary intelligence (Sections §17 onwards)** orchestrates weather/NDVI staging, hive log synthesis, and two tiers of ML models (sklearn HistGradientBoosting + PyTorch temporal CNN/GRU) to estimate hive stress / occupancy risk.

Every block includes deterministic filesystem staging and writes intermediate products to `/kaggle/working` or `content/` so the research report can quote exact metrics while Kaggle submissions remain GPU safe.


In [77]:
!pip install -q earthengine-api ipyleaflet ipywidgets keras-tuner librosa tqdm

In [78]:
import os
import shutil
from pathlib import Path

PROJECT_ROOT = Path.cwd()
DEFAULT_CONTENT = PROJECT_ROOT / "content"
KAGGLE_WORKING = Path("/kaggle/working")

if DEFAULT_CONTENT.exists():
    CONTENT_ROOT = DEFAULT_CONTENT.resolve()
else:
    CONTENT_ROOT = (KAGGLE_WORKING / "content").resolve()
    CONTENT_ROOT.mkdir(parents=True, exist_ok=True)

os.environ["MERGED_CONTENT_ROOT"] = str(CONTENT_ROOT)
MAIN_DATA_DIR = (CONTENT_ROOT / "main-data")
MAIN_DATA_DIR.mkdir(parents=True, exist_ok=True)

KAGGLE_INPUT_ROOT = Path("/kaggle/input")

def _stage_dataset(keyword, target_subdir):
    if not KAGGLE_INPUT_ROOT.exists():
        return None
    matches = [p for p in KAGGLE_INPUT_ROOT.iterdir() if keyword in p.name.lower()]
    if not matches:
        print(f"[setup] Kaggle input dataset containing '{keyword}' not found.")
        return None
    source = matches[0]
    target = CONTENT_ROOT / target_subdir
    shutil.rmtree(target, ignore_errors=True)
    shutil.copytree(source, target, dirs_exist_ok=True)
    print(f"[setup] Staged {source.name} -> {target}")
    return target

def _maybe_stage(keyword, subdir):
    try:
        _stage_dataset(keyword, subdir)
    except Exception as exc:
        print(f"[setup] Skipping auto-stage for {keyword}: {exc}")

_maybe_stage("beehive", "beehive_audio")
_maybe_stage("makueni", "main-data")

print(f"CONTENT_ROOT -> {CONTENT_ROOT}")


[setup] Staged beehive-audio-dataset-with-queen-and-without-queen -> /kaggle/working/content/beehive_audio
[setup] Staged makueni-ndvi-2008-2025-csv -> /kaggle/working/content/main-data
CONTENT_ROOT -> /kaggle/working/content


In [79]:
import calendar
import datetime as dt
import gc
import io
import json
import math
import os
import time
import warnings
from pathlib import Path
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    RocCurveDisplay
)
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import requests
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 4)
CONTENT_ROOT = Path(os.environ["MERGED_CONTENT_ROOT"])
MAIN_DATA_DIR = CONTENT_ROOT / "main-data"
from pathlib import Path
FIGURE_DIR = Path('artifacts/figures')
FIGURE_DIR.mkdir(parents=True, exist_ok=True)
print('Saving figures and tables to', FIGURE_DIR.resolve())



In [ ]:
from pathlib import Path
FIGURE_DIR = Path('artifacts/figures')
FIGURE_DIR.mkdir(parents=True, exist_ok=True)
print('Saving figures and tables to', FIGURE_DIR.resolve())


In [ ]:
FIGURE_DIR = Path('/kaggle/working/figures')
FIGURE_DIR.mkdir(parents=True, exist_ok=True)
print('Saving figures and tables to', FIGURE_DIR)


## Queen Bee Acoustic Detection Pipeline

### Acoustic Dataset Staging & Lineage

The queen-bee classifier is trained from the Kaggle dataset `harshkumar1711/beehive-audio-dataset-with-queen-and-without-queen`. We avoid copying raw WAVs into writable storage unless needed; instead, `_discover_audio_dataset` crawls the mounted `/kaggle/input` tree, validates the folder structure, and exposes canonical paths for the `QueenBee Present`, `QueenBee Absent`, and `External Noise` subsets. This guarantees that every spectrogram (and therefore every model checkpoint) can be traced back to a known dataset version, satisfying the reproducibility requirement in the BeeUnity methodology.


In [80]:
from pathlib import Path

def _discover_audio_dataset(content_root: Path) -> Path:
    search_root = Path("/kaggle/input/beehive-audio-dataset-with-queen-and-without-queen")
    if not search_root.exists():
        raise FileNotFoundError(
            "Dataset not staged. Attach Kaggle dataset "
            "'harshkumar1711/beehive-audio-dataset-with-queen-and-without-queen'."
        )

    for candidate in sorted(search_root.rglob("Dataset")):
        if (candidate / "Bee Hive Audios").exists():
            return candidate

    raise FileNotFoundError("Could not locate 'Dataset/Bee Hive Audios'.")

# Discover dataset (READ-ONLY)
AUDIO_DATASET_ROOT = _discover_audio_dataset(None)

BEEHIVE_AUDIO_DIR = next(AUDIO_DATASET_ROOT.glob("**/Bee Hive Audios"))
QUEEN_PRESENT_DIR = BEEHIVE_AUDIO_DIR / "QueenBee Present"
QUEEN_ABSENT_DIR = BEEHIVE_AUDIO_DIR / "QueenBee Absent"
EXTERNAL_DIR = AUDIO_DATASET_ROOT / "External Noise"

# WRITEABLE spectrogram directory
SPECTROGRAM_DIR = Path("/kaggle/working/spectrograms")
SPECTROGRAM_PRESENT = SPECTROGRAM_DIR / "present"
SPECTROGRAM_ABSENT = SPECTROGRAM_DIR / "absent"
SPECTROGRAM_EXTERNAL = SPECTROGRAM_DIR / "external"

for path in [SPECTROGRAM_PRESENT, SPECTROGRAM_ABSENT, SPECTROGRAM_EXTERNAL]:
    path.mkdir(parents=True, exist_ok=True)

print("Audio dataset root (read-only):", AUDIO_DATASET_ROOT)
print("Spectrogram cache (writable):", SPECTROGRAM_DIR)


Audio dataset root (read-only): /kaggle/input/beehive-audio-dataset-with-queen-and-without-queen/Dataset
Spectrogram cache (writable): /kaggle/working/spectrograms


In [81]:
try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
    strategy = tf.distribute.TPUStrategy(tpu_resolver)
    ACCELERATOR = "TPU"
except (ValueError, tf.errors.NotFoundError):
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        for gpu in gpus:
            try:
                tf.config.experimental.set_memory_growth(gpu, True)
            except Exception:
                pass
        # Default to single-replica strategy for Kaggle GPU stability
        strategy = tf.distribute.get_strategy()
        ACCELERATOR = "GPU"
    else:
        strategy = tf.distribute.get_strategy()
        ACCELERATOR = "CPU"

print(f"Using {ACCELERATOR} via {strategy.__class__.__name__}")


Using GPU via _DefaultDistributionStrategy


### Audio Conditioning & Spectrogram Cache

To stabilize CNN training we transform each WAV into a fixed 3-second mono clip sampled at 22.05 kHz. The `preprocess_and_save_spectrogram` routine trims silence, enforces constant-length padding, normalizes amplitude, and renders a 128×128 mel-spectrogram using librosa. Spectrograms land under `/kaggle/working/spectrograms/<class>/` and the generators only touch PNGs, eliminating expensive audio decoding during model fit. Progress-aware helpers (e.g., `_compute_progress`) let reruns skip already materialized windows so Kaggle GPU runtime stays within budget.


In [82]:
SAMPLE_RATE = 22050
DURATION = 3
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

librosa.cache.clear()
plt.switch_backend("Agg")

def preprocess_and_save_spectrogram(audio_path: Path, output_image_path: Path, sr=SAMPLE_RATE, duration=DURATION):
    try:
        y, _ = librosa.load(audio_path, sr=sr)
        y, _ = librosa.effects.trim(y)
        y = librosa.to_mono(y) if y.ndim > 1 else y
        y = librosa.util.normalize(y)

        expected_samples = sr * duration
        if len(y) < expected_samples:
            y = np.pad(y, (0, expected_samples - len(y)), mode="constant")
        else:
            y = y[:expected_samples]

        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
        mel_db = librosa.power_to_db(mel_spec, ref=np.max)

        plt.figure(figsize=(2, 2), dpi=64)
        librosa.display.specshow(mel_db, sr=sr, cmap="magma")
        plt.axis("off")
        output_image_path.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(output_image_path, bbox_inches="tight", pad_inches=0)
        plt.close()
    except Exception as exc:
        print(f"[spectrogram] Failed on {audio_path}: {exc}")

def _compute_progress(files, output_dir: Path):
    total = len(files)
    processed = sum((output_dir / f"{Path(f).stem}.png").exists() for f in files)
    return total, processed

def process_audio_folder(input_dir: Path, output_dir: Path, desc: str):
    if not input_dir.exists():
        print(f"[spectrogram] {input_dir} missing, skipping {desc}.")
        return
    wav_files = sorted([f for f in input_dir.iterdir() if f.suffix.lower() == ".wav"])
    total, processed = _compute_progress([f.name for f in wav_files], output_dir)
    with tqdm(total=total, initial=processed, desc=desc, unit="file") as pbar:
        for wav_path in wav_files:
            out_path = output_dir / f"{wav_path.stem}.png"
            if out_path.exists():
                pbar.update(1)
                continue
            preprocess_and_save_spectrogram(wav_path, out_path)
            gc.collect()
            pbar.update(1)

def process_external_folder(input_dir: Path, output_dir: Path):
    if not input_dir.exists():
        print("[spectrogram] External noise folder missing, skipping.")
        return
    audio_paths = []
    for root, _, files in os.walk(input_dir):
        audio_paths += [Path(root) / f for f in files if f.lower().endswith(".wav")]
    with tqdm(total=len(audio_paths), desc="External noise", unit="file") as pbar:
        for wav_path in audio_paths:
            out_path = output_dir / f"{wav_path.stem}.png"
            if out_path.exists():
                pbar.update(1)
                continue
            preprocess_and_save_spectrogram(wav_path, out_path)
            pbar.update(1)


[Memory(location=None)]: Flushing completely the cache


In [83]:
process_audio_folder(QUEEN_PRESENT_DIR, SPECTROGRAM_PRESENT, "QueenBee Present")
process_audio_folder(QUEEN_ABSENT_DIR, SPECTROGRAM_ABSENT, "QueenBee Absent")
process_external_folder(EXTERNAL_DIR, SPECTROGRAM_EXTERNAL)

QueenBee Present: 8000file [00:00, 77319.70file/s]             
QueenBee Absent: 4000file [00:00, 80590.73file/s]             
External noise: 100%|██████████| 2000/2000 [00:00<00:00, 59606.97file/s]


In [84]:
def count_pngs(folder: Path):
    return len([f for f in folder.glob("*.png")])

class_labels = ["present", "absent", "external"]
counts = [
    count_pngs(SPECTROGRAM_PRESENT),
    count_pngs(SPECTROGRAM_ABSENT),
    count_pngs(SPECTROGRAM_EXTERNAL),
]

plt.figure(figsize=(6, 4))
bars = plt.bar(class_labels, counts, color=["sienna", "peru", "gray"], edgecolor="black")
plt.ylim(0, max(counts) * 1.1 if counts else 10)
plt.title("Spectrogram Count per Class")
plt.ylabel("Images")
for bar in bars:
    y = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, y + max(1, y ** 0.5), int(y), ha="center", va="bottom")
plt.show()

print(dict(zip(class_labels, counts)))


{'present': 4000, 'absent': 2000, 'external': 2000}


### Stratified DataFrames, Augmentation, and Class Weights

The original ImageDataGenerator split approach caused leakage between validation/test folds. We now build a pandas catalog of every spectrogram file, stratify it into train/val/test via `train_test_split`, and feed `flow_from_dataframe` generators. Light-weight augmentations (flip + shifts) only touch the training subset. Class imbalance (present:absent:external = 4000:2000:2000) is mitigated through `compute_class_weight` and a custom `SparseClassRecall` metric that explicitly tracks recall on the underrepresented `absent` class; both feed into every Keras fit/tuning call so the notebook’s metrics align with the research objective of catching queen loss events.


In [85]:
IMG_SIZE = (128, 128)
BASE_BATCH_SIZE = 32
BATCH_SIZE = BASE_BATCH_SIZE  # Keep per-device batch size stable on Kaggle
SEED = 42

spectro_records = []
for class_dir in sorted(SPECTROGRAM_DIR.iterdir()):
    if class_dir.is_dir():
        label = class_dir.name
        for img_path in class_dir.glob("*.png"):
            spectro_records.append({"filepath": str(img_path), "label": label})

if not spectro_records:
    raise RuntimeError("No spectrograms were generated; run preprocessing above first.")

spectro_df = pd.DataFrame(spectro_records)
CLASS_NAMES = sorted(spectro_df["label"].unique())

train_df, temp_df = train_test_split(
    spectro_df,
    test_size=0.4,
    stratify=spectro_df["label"],
    random_state=SEED
)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["label"],
    random_state=SEED
)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.05,
    height_shift_range=0.05
)
eval_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    classes=CLASS_NAMES,
    shuffle=True,
    seed=SEED
)

val_gen = eval_datagen.flow_from_dataframe(
    val_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    classes=CLASS_NAMES,
    shuffle=False,
    seed=SEED
)

test_gen = eval_datagen.flow_from_dataframe(
    test_df,
    x_col="filepath",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    classes=CLASS_NAMES,
    shuffle=False,
    seed=SEED
)

raw_class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array(CLASS_NAMES),
    y=train_df["label"]
)
CLASS_WEIGHTS = {
    train_gen.class_indices[label]: weight for label, weight in zip(CLASS_NAMES, raw_class_weights)
}
print("Class indices:", train_gen.class_indices)
print("Class weights:", CLASS_WEIGHTS)

ABSENT_CLASS_INDEX = train_gen.class_indices["absent"]

class SparseClassRecall(tf.keras.metrics.Metric):
    def __init__(self, class_id, name="sparse_class_recall", **kwargs):
        super().__init__(name=name, **kwargs)
        self.class_id = class_id
        self.true_positives = self.add_weight(name="tp", initializer="zeros")
        self.false_negatives = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(tf.reshape(y_true, [-1]), tf.int32)
        y_pred = tf.cast(tf.argmax(y_pred, axis=-1), tf.int32)
        class_mask = tf.cast(tf.equal(y_true, self.class_id), self.dtype)
        pred_mask = tf.cast(tf.equal(y_pred, self.class_id), self.dtype)
        if sample_weight is None:
            weights = tf.ones_like(class_mask)
        else:
            weights = tf.cast(tf.reshape(sample_weight, [-1]), self.dtype)
            weights = tf.broadcast_to(weights, tf.shape(class_mask))
        weighted_mask = class_mask * weights
        tp = tf.reduce_sum(pred_mask * weighted_mask)
        fn = tf.reduce_sum((1.0 - pred_mask) * weighted_mask)
        self.true_positives.assign_add(tp)
        self.false_negatives.assign_add(fn)

    def get_config(self):
        config = super().get_config()
        config.update({"class_id": int(self.class_id)})
        return config

    def result(self):
        return tf.math.divide_no_nan(self.true_positives, self.true_positives + self.false_negatives)

    def reset_states(self):
        self.true_positives.assign(0.0)
        self.false_negatives.assign(0.0)

def make_absent_recall(name="recall_absent"):
    return SparseClassRecall(class_id=ABSENT_CLASS_INDEX, name=name)



Found 4800 validated image filenames belonging to 3 classes.
Found 1600 validated image filenames belonging to 3 classes.
Found 1600 validated image filenames belonging to 3 classes.
Class indices: {'absent': 0, 'external': 1, 'present': 2}
Class weights: {0: np.float64(1.3333333333333333), 1: np.float64(1.3333333333333333), 2: np.float64(0.6666666666666666)}


### Baseline CNN Training Plan

The baseline network is intentionally compact so it trains quickly on Kaggle GPUs yet captures salient spectral patterns: three Conv-BN-Pool stages followed by GAP and a 64-unit dense head. Training runs under the selected `strategy` with class weights + early stopping keyed to `val_recall_absent` to bias the model toward correctly flagging queen-absent clips. This baseline establishes the minimum viable performance before hyperparameter search.


In [86]:
from tensorflow.keras.callbacks import EarlyStopping

def build_baseline_model():
    model = models.Sequential([
        layers.Conv2D(32, 3, activation="relu", padding="same", input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.GlobalAveragePooling2D(),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(3, activation="softmax"),
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy", make_absent_recall()]
    )
    return model

with strategy.scope():
    baseline_model = build_baseline_model()

baseline_callbacks = [
    EarlyStopping(monitor="val_recall_absent", mode="max", patience=3, restore_best_weights=True)
]

baseline_history = baseline_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    class_weight=CLASS_WEIGHTS,
    callbacks=baseline_callbacks
)


Epoch 1/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 26s 144ms/step - accuracy: 0.7123 - loss: 0.6749 - recall_absent: 0.7678 - val_accuracy: 0.5294 - val_loss: 1.1168 - val_recall_absent: 0.0000e+00
Epoch 2/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 140ms/step - accuracy: 0.8961 - loss: 0.2786 - recall_absent: 0.9125 - val_accuracy: 0.3481 - val_loss: 1.7710 - val_recall_absent: 0.0000e+00
Epoch 3/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 140ms/step - accuracy: 0.9316 - loss: 0.1825 - recall_absent: 0.9241 - val_accuracy: 0.2500 - val_loss: 5.2762 - val_recall_absent: 0.0000e+00
Epoch 4/20
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 140ms/step - accuracy: 0.9405 - loss: 0.1604 - recall_absent: 0.9390 - val_accuracy: 0.7387 - val_loss: 2.5434 - val_recall_absent: 0.0000e+00


### KerasTuner Hyperband Search & Fine-Tune

Hyperparameter tuning explores filter widths, dense units, dropout, and optimizer choice via `kt.Hyperband`, again optimizing `val_recall_absent`. The tuner runs outside the distribution `strategy` scope (per TensorFlow guidance) while the search/ fine-tune phases inherit the same class weights + early stopping regime as the baseline. The best trial is persisted as a `.keras` artifact under `/kaggle/working` for downstream deployment / report inclusion.


In [87]:
from tensorflow.keras.callbacks import EarlyStopping
from pathlib import Path

def build_tunable_model(hp):
    model = models.Sequential([
        layers.Conv2D(
            hp.Choice("conv1", [32, 64]), 3,
            activation="relu", padding="same",
            input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
        ),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(hp.Choice("conv2", [64, 128]), 3, activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(hp.Choice("conv3", [128, 256]), 3, activation="relu", padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.GlobalAveragePooling2D(),
        layers.Dense(hp.Int("dense_units", 64, 128, step=32), activation="relu"),
        layers.Dropout(hp.Float("dropout", 0.3, 0.6, step=0.1)),
        layers.Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=hp.Choice("optimizer", ["adam", "nadam"]),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy", make_absent_recall()]
    )
    return model


# Strategy ONLY for tuner creation
with strategy.scope():
    tuner = kt.Hyperband(
        build_tunable_model,
        objective=kt.Objective("val_recall_absent", direction="max"),
        max_epochs=15,
        factor=3,
        directory="/kaggle/working/queenbee_tuning",
        project_name="queenbee_cnn"
    )

stopper = EarlyStopping(
    monitor="val_recall_absent",
    mode="max",
    patience=3,
    restore_best_weights=True
)

# Search OUTSIDE strategy scope
tuner.search(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=CLASS_WEIGHTS,
    callbacks=[stopper]
)

# NO strategy scope here
best_model = tuner.get_best_models(num_models=1)[0]

fine_tune_history = best_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=CLASS_WEIGHTS,
    callbacks=[stopper]
)

# Writable save path
best_model_path = Path("/kaggle/working/queenbee_final_tuned_model.keras")
best_model.save(best_model_path)

print("Saved tuned model to", best_model_path)


Reloading Tuner from /kaggle/working/queenbee_tuning/queenbee_cnn/tuner0.json
Epoch 1/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 27s 146ms/step - accuracy: 0.8976 - loss: 0.3386 - recall_absent: 0.8787 - val_accuracy: 0.5306 - val_loss: 0.9954 - val_recall_absent: 0.9900
Epoch 2/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 142ms/step - accuracy: 0.9532 - loss: 0.1079 - recall_absent: 0.9564 - val_accuracy: 0.9081 - val_loss: 0.2799 - val_recall_absent: 0.9575
Epoch 3/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 142ms/step - accuracy: 0.9639 - loss: 0.0746 - recall_absent: 0.9640 - val_accuracy: 0.9381 - val_loss: 0.1628 - val_recall_absent: 0.8500
Epoch 4/15
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 143ms/step - accuracy: 0.9630 - loss: 0.0877 - recall_absent: 0.9620 - val_accuracy: 0.5750 - val_loss: 1.4998 - val_recall_absent: 0.2425
Saved tuned model to /kaggle/working/queenbee_final_tuned_model.keras


In [88]:
from tensorflow.keras.models import load_model

model_for_eval = load_model(
    best_model_path,
    custom_objects={"SparseClassRecall": SparseClassRecall}
)


### Probability Calibration & Threshold Selection

Raw softmax scores tend to collapse onto the majority `present` class. After loading the tuned CNN we perform two evaluation modes: standard argmax and calibrated predictions. Validation probabilities drive per-class precision-recall curves, from which we select F1-optimal thresholds. Those calibrated thresholds are then applied to the held-out test generator, yielding confusion matrices, detailed classification reports, and macro ROC/PR AUC metrics that the manuscript can cite when describing queen-state detection performance.


In [89]:
def run_inference(model, generator):
    generator.reset()
    y_prob = model.predict(generator, verbose=1)
    y_true = generator.classes
    return y_prob, y_true


def derive_thresholds(y_true, y_prob, class_names):
    y_true_oh = tf.keras.utils.to_categorical(y_true, num_classes=len(class_names))
    thresholds = {}
    for idx, name in enumerate(class_names):
        precision, recall, thresh = precision_recall_curve(y_true_oh[:, idx], y_prob[:, idx])
        if thresh.size == 0:
            thresholds[name] = 0.5
            continue
        f1 = 2 * precision * recall / np.clip(precision + recall, 1e-8, None)
        best_idx = np.nanargmax(f1)
        thresholds[name] = float(thresh[min(best_idx, len(thresh) - 1)])
    return thresholds


def predict_with_thresholds(y_prob, class_names, thresholds):
    calibrated = []
    for row in y_prob:
        chosen_idx = None
        chosen_score = -1.0
        for idx, name in enumerate(class_names):
            threshold = thresholds.get(name, 0.5)
            if row[idx] >= threshold and row[idx] > chosen_score:
                chosen_idx = idx
                chosen_score = row[idx]
        if chosen_idx is None:
            chosen_idx = int(np.argmax(row))
        calibrated.append(chosen_idx)
    return np.array(calibrated)


def summarize_metrics(y_true, y_pred, label):
    return {
        "Mode": label,
        "Accuracy": accuracy_score(y_true, y_pred),
        "Macro Precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "Macro Recall": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "Macro F1": f1_score(y_true, y_pred, average="macro", zero_division=0)
    }

val_prob, val_true = run_inference(model_for_eval, val_gen)
class_names = list(test_gen.class_indices.keys())
thresholds = derive_thresholds(val_true, val_prob, class_names)
print("Calibrated probability thresholds:")
for name in class_names:
    print(f"  {name}: {thresholds[name]:.3f}")

metrics = []
test_prob, test_true = run_inference(model_for_eval, test_gen)
default_pred = np.argmax(test_prob, axis=1)
calibrated_pred = predict_with_thresholds(test_prob, class_names, thresholds)

metrics_table = pd.DataFrame([
    summarize_metrics(test_true, default_pred, "Argmax"),
    summarize_metrics(test_true, calibrated_pred, "Calibrated")
])
display(metrics_table)
metrics_table_path = FIGURE_DIR / "acoustic_metrics_table.csv"
metrics_table.to_csv(metrics_table_path, index=False)
print("Saved acoustic metrics table ->", metrics_table_path)

cm = confusion_matrix(test_true, calibrated_pred)
cm_fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names, ax=ax)
ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_title("Confusion Matrix (Calibrated)")
cm_path = FIGURE_DIR / "acoustic_confusion_matrix.png"
cm_fig.savefig(cm_path, dpi=300, bbox_inches="tight")
plt.show()
print("Saved acoustic confusion matrix ->", cm_path)

report_text = classification_report(test_true, calibrated_pred, target_names=class_names, zero_division=0)
print("Calibrated classification report:
", report_text)
report_path = FIGURE_DIR / "acoustic_classification_report.txt"
report_path.write_text(report_text)

roc_auc = roc_auc_score(
    pd.get_dummies(test_true, drop_first=False).values,
    test_prob,
    average="macro",
    multi_class="ovr"
)
pr_auc = average_precision_score(
    pd.get_dummies(test_true, drop_first=False).values,
    test_prob,
    average="macro"
)
auc_path = FIGURE_DIR / "acoustic_auc_summary.json"
auc_path.write_text(json.dumps({"roc_auc": float(roc_auc), "pr_auc": float(pr_auc)}, indent=2))
print(f"ROC-AUC: {roc_auc:.4f} | PR-AUC: {pr_auc:.4f}")
print("Saved acoustic AUC summary ->", auc_path)



50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step
Calibrated probability thresholds:
  absent: 0.898
  external: 0.779
  present: 0.032
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step


,Mode,Accuracy,Macro Precision,Macro Recall,Macro F1
0,Argmax,0.526875,0.695540,0.681667,0.501093
1,Calibrated,0.946875,0.937807,0.957083,0.946250


Calibrated classification report:               precision    recall  f1-score   support

      absent       0.89      0.97      0.93       400
    external       0.94      0.98      0.96       400
     present       0.99      0.92      0.95       800

    accuracy                           0.95      1600
   macro avg       0.94      0.96      0.95      1600
weighted avg       0.95      0.95      0.95      1600

ROC-AUC: 0.9813 | PR-AUC: 0.9667


In [90]:
SR = 22050

def audio_to_spectrogram_image(audio_path: Path):
    y, sr = librosa.load(audio_path, sr=SR)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, n_fft=2048, hop_length=512)
    S_dB = librosa.power_to_db(S, ref=np.max)

    fig = plt.figure(figsize=(2, 2), dpi=64)
    librosa.display.specshow(S_dB, sr=sr, cmap="magma")
    plt.axis("off")

    buf = io.BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight", pad_inches=0)
    plt.close(fig)
    buf.seek(0)

    img = Image.open(buf).convert("RGB").resize(IMG_SIZE)
    img_array = np.array(img, dtype=np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def visualize_audio_prediction(audio_path: Path, model):
    mel_input = audio_to_spectrogram_image(audio_path)
    prediction = model.predict(mel_input)
    class_names = list(test_gen.class_indices.keys())
    pred_idx = int(np.argmax(prediction))
    confidence = float(np.max(prediction))

    y, sr = librosa.load(audio_path, sr=SR)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_db = librosa.power_to_db(mel, ref=np.max)

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    times = np.linspace(0, len(y)/sr, len(y))
    axes[0,0].plot(times, y)
    axes[0,0].set_title("Waveform")

    img = axes[0,1].imshow(mel_db, aspect="auto", origin="lower", cmap="magma")
    axes[0,1].set_title("Mel Spectrogram")
    plt.colorbar(img, ax=axes[0,1], fraction=0.046, pad=0.04)

    axes[1,0].bar(class_names, prediction[0], color="teal")
    axes[1,0].set_ylim(0, 1)
    axes[1,0].set_title("Prediction Probabilities")

    axes[1,1].axis("off")
    axes[1,1].text(0.1, 0.5, f"Predicted: {class_names[pred_idx]}\nConfidence: {confidence:.2%}", fontsize=14)

    plt.tight_layout()
    plt.show()

    return {"prediction": class_names[pred_idx], "confidence": confidence}

sample_audio = next(QUEEN_PRESENT_DIR.glob('*.wav'))
visualize_audio_prediction(sample_audio, model_for_eval)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step


{'prediction': 'external', 'confidence': 0.6416568160057068}

## Makueni Climate-Informed Forecasting

We now project yield and occupancy directly from climate/NDVI sequences using Würzburg-pretrained models.


## Makueni Apiary Intelligence Pipeline

The second half of BeeUnity focuses on environmental + hive telemetry analytics for Makueni County. Users can optionally pick a geometry via ipyleaflet; however, Kaggle’s environment rarely ships the `jupyter-leaflet` extension, so we default to fixed coordinates while preserving the widget wiring for local notebooks. This section obeys Kaggle’s outbound-network policy via the `ENABLE_REMOTE_CALLS` flag and falls back to cached CSV exports inside `content/main-data/`.


In [103]:
DEFAULT_CENTER = (-1.8048, 37.62)
ENABLE_LEAFLET_WIDGETS = False  # Set True only if jupyter-leaflet widgets are installed.

try:
    import ipywidgets as widgets
    from ipyleaflet import Map, Marker, DrawControl, basemaps
except Exception:
    print("ipyleaflet not available; using default coordinates.")
    lat_widget = lon_widget = geometry_widget = None
else:
    lat_widget = widgets.FloatText(value=DEFAULT_CENTER[0], description="Latitude", step=0.0001)
    lon_widget = widgets.FloatText(value=DEFAULT_CENTER[1], description="Longitude", step=0.0001)
    geometry_widget = widgets.Textarea(
        value="",
        description="Geometry",
        placeholder="Draw a polygon/rectangle on the map.",
        layout=widgets.Layout(width="100%", height="140px"),
        disabled=True,
    )

    leaflet_map = Map(center=DEFAULT_CENTER, zoom=8, basemap=basemaps.OpenStreetMap.Mapnik, scroll_wheel_zoom=True)
    marker = Marker(location=DEFAULT_CENTER, draggable=True)
    leaflet_map.add_layer(marker)

    draw_control = DrawControl(
        polygon={"shapeOptions": {"color": "#2563eb", "weight": 2, "fillOpacity": 0.2}},
        rectangle={"shapeOptions": {"color": "#f97316", "weight": 2, "fillOpacity": 0.15}},
        circle={},
        circlemarker={},
        polyline={},
    )
    leaflet_map.add_control(draw_control)

    def _update_marker(change):
        marker.location = (lat_widget.value, lon_widget.value)

    lat_widget.observe(_update_marker, names="value")
    lon_widget.observe(_update_marker, names="value")

    display(widgets.HBox([lat_widget, lon_widget]))
    display(geometry_widget)
    display(leaflet_map)

lat_widget_available = 'lat_widget' in globals() and lat_widget is not None
lon_widget_available = 'lon_widget' in globals() and lon_widget is not None

if lat_widget_available and lon_widget_available:
    latitude = float(lat_widget.value)
    longitude = float(lon_widget.value)
else:
    latitude, longitude = DEFAULT_CENTER
    print("Using default coordinates:", DEFAULT_CENTER)

selected_geometry_geojson = globals().get('selected_geometry_geojson')


Textarea(value='', description='Geometry', disabled=True, layout=Layout(height='140px', width='100%'), placeho…

Map(center=[-1.8048, 37.62], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

### Weather/NDVI Acquisition Strategy

We scope the modeling window via `normalize_date_string`, clamp requests to the latest Open-Meteo archive availability, and split long ranges into 365-day chunks. When `ENABLE_REMOTE_CALLS` is false (the Kaggle default), we load pre-exported weather and NDVI CSVs staged under `content/main-data/`. When high-trust compute is available, the notebook can re-fetch ERA5/Open-Meteo and MODIS NDVI slices, persisting them with consistent schemas so report figures remain reproducible.


In [104]:
import ee

raw_start_date = "2008-01-01"
raw_end_date = "2025-12-05"
timezone = "Africa/Nairobi"

def normalize_date_string(d: str) -> dt.date:
    parts = d.split("-")
    if len(parts) != 3:
        raise ValueError("Date must be YYYY-MM-DD")
    y, m, day = [int(p) for p in parts]
    m = max(1, min(12, m))
    last_day = calendar.monthrange(y, m)[1]
    day = max(1, min(last_day, day))
    return dt.date(y, m, day)

start_date = normalize_date_string(raw_start_date)
end_date = normalize_date_string(raw_end_date)

today = dt.date.today()
api_latest = dt.date(2025, 12, 20)
max_allowed = min(today, api_latest)

if end_date > max_allowed:
    print(f"Clamping end_date {end_date} -> {max_allowed}")
    end_date = max_allowed
if start_date > end_date:
    raise ValueError("start_date must be before end_date")

print("Using date range:", start_date, "→", end_date)


Using date range: 2008-01-01 → 2025-12-05


In [107]:
ENABLE_REMOTE_CALLS = True  # Kaggle notebooks typically block outbound internet.

def split_date_range(start: dt.date, end: dt.date, max_days: int = 365):
    chunks = []
    current = start
    while current <= end:
        chunk_end = min(end, current + dt.timedelta(days=max_days - 1))
        chunks.append((current, chunk_end))
        current = chunk_end + dt.timedelta(days=1)
    return chunks

def fetch_chunk(lat, lon, sdate: dt.date, edate: dt.date, timezone="Africa/Nairobi", max_retries=3, backoff=2):
    base = "https://archive-api.open-meteo.com/v1/archive"
    daily_vars = ",".join([
        "temperature_2m_max",
        "temperature_2m_min",
        "temperature_2m_mean",
        "precipitation_sum",
        "relative_humidity_2m_mean",
        "wind_speed_10m_max",
        "cloudcover_mean"
    ])
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": sdate.strftime("%Y-%m-%d"),
        "end_date": edate.strftime("%Y-%m-%d"),
        "daily": daily_vars,
        "timezone": timezone
    }
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(base, params=params, timeout=30)
            resp.raise_for_status()
            payload = resp.json()
            if "daily" not in payload or "time" not in payload["daily"]:
                raise ValueError("API response missing expected fields.")
            return payload
        except Exception as exc:
            print(f"Attempt {attempt} failed: {exc}")
            if attempt == max_retries:
                raise
            time.sleep(backoff ** attempt)


In [108]:
weather_csv = MAIN_DATA_DIR / "makueni_weather_2008_2025.csv"
chunks = split_date_range(start_date, end_date, max_days=365)

if ENABLE_REMOTE_CALLS:
    dfs = []
    for s, e in chunks:
        payload = fetch_chunk(latitude, longitude, s, e, timezone=timezone)
        daily = payload["daily"]
        df_chunk = pd.DataFrame({
            "date": daily["time"],
            "temp_max": daily.get("temperature_2m_max"),
            "temp_min": daily.get("temperature_2m_min"),
            "temp_mean": daily.get("temperature_2m_mean"),
            "humidity_mean": daily.get("relative_humidity_2m_mean"),
            "rainfall_mm": daily.get("precipitation_sum"),
            "wind_speed_max": daily.get("wind_speed_10m_max"),
            "cloud_cover_percent": daily.get("cloudcover_mean"),
        })
        dfs.append(df_chunk)
        time.sleep(1)
    weather_df = pd.concat(dfs, ignore_index=True)
    weather_df["date"] = pd.to_datetime(weather_df["date"])
    weather_df.sort_values("date", inplace=True)
    weather_df.to_csv(weather_csv, index=False)
    print("Fetched and saved weather CSV to", weather_csv)
else:
    if weather_csv.exists():
        weather_df = pd.read_csv(weather_csv, parse_dates=["date"])
        print(f"Loaded cached weather data from {weather_csv}")
    else:
        raise FileNotFoundError(f"{weather_csv} not found; enable ENABLE_REMOTE_CALLS to regenerate.")


Fetched and saved weather CSV to /kaggle/working/content/main-data/makueni_weather_2008_2025.csv


In [109]:
ENABLE_REMOTE_CALLS = False  # Kaggle notebooks typically block outbound internet.
ndvi_csv = "/kaggle/input/makueni-ndvi-2008-2025-csv/makueni_ndvi_2008_2025.csv"

if ENABLE_REMOTE_CALLS:
    try:
        ee.Initialize()
    except Exception:
        print("Authenticating with Earth Engine...")
        ee.Authenticate()
        ee.Initialize()

    point = ee.Geometry.Point([longitude, latitude])
    modis = ee.ImageCollection("MODIS/061/MOD13Q1").select("NDVI").filterDate(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")).filterBounds(point)

    def extract_ndvi(image):
        mean = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=point, scale=250).get("NDVI")
        date = image.date().format("YYYY-MM-dd")
        return ee.Feature(None, {"date": date, "ndvi_mean": mean})

    ndvi_fc = modis.map(extract_ndvi).getInfo()
    records = [f["properties"] for f in ndvi_fc["features"]]
    ndvi_df = pd.DataFrame(records)
    ndvi_df["date"] = pd.to_datetime(ndvi_df["date"])
    ndvi_df["ndvi_mean"] = ndvi_df["ndvi_mean"].astype(float) / 10000
    ndvi_df.to_csv(ndvi_csv, index=False)
    print("Fetched NDVI and saved to", ndvi_csv)
else:
    if ndvi_csv:
        ndvi_df = pd.read_csv(ndvi_csv, parse_dates=["date"])
        print(f"Loaded cached NDVI data from {ndvi_csv}")
    else:
        raise FileNotFoundError(f"{ndvi_csv} not found; enable ENABLE_REMOTE_CALLS to regenerate.")


Loaded cached NDVI data from /kaggle/input/makueni-ndvi-2008-2025-csv/makueni_ndvi_2008_2025.csv


In [110]:
merged = weather_full.copy()
merged_path = MAIN_DATA_DIR / "makueni_climate_features.csv"
merged.to_csv(merged_path, index=False)
print("Saved climate feature table ->", merged_path)
merged.head()


Merged weather+NDVI -> /kaggle/working/content/main-data/makueni_weather_ndvi_2008_2025.csv


,date,temp_max,temp_min,temp_mean,humidity_mean,rainfall_mm,wind_speed_max,cloud_cover_percent,ndvi_mean
0,2008-01-01,24.9,16.4,20.3,74,1.2,15.1,53,0.6805
1,2008-01-02,25.8,14.1,20.2,71,0.8,14.3,19,NaN
2,2008-01-03,27.2,15.2,21.3,65,0.0,12.8,11,NaN
3,2008-01-04,27.6,15.4,22.2,63,0.1,12.2,26,NaN
4,2008-01-05,27.3,15.2,21.0,75,2.9,13.1,58,NaN


In [111]:
weather_full = df_merged.copy()
print('Weather+NDVI rows:', weather_full.shape)


Weather+NDVI rows: (6549, 9)


In [112]:
df_month = df_merged.set_index("date").resample("ME").agg({
    "rainfall_mm": "sum",
    "temp_mean": "mean",
    "humidity_mean": "mean",
    "ndvi_mean": "mean"
}).reset_index()

fig, axes = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
axes[0].plot(df_month["date"], df_month["rainfall_mm"], marker="o")
axes[0].set_title("Monthly Rainfall (mm)")

axes[1].plot(df_month["date"], df_month["temp_mean"], marker="o", color="tomato")
axes[1].set_title("Monthly Mean Temperature (°C)")

axes[2].plot(df_month["date"], df_month["ndvi_mean"], marker="o", color="green")
axes[2].set_title("Monthly NDVI Mean")

for ax in axes:
    ax.grid(True, alpha=0.3)
    ax.set_ylabel("Value")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Hive Telemetry & Synthetic Augmentation

Direct hive telemetry from community partners is still sparse, so the notebook synthesizes weekly hive logs per hive id (Honey yield, Varroa %, hive weight, brood area, stress events). The generator preserves realistic ranges/variances and encodes queen age metadata so the downstream models can learn temporal drift patterns. When actual CSV exports (`hive_logs_2008_2025.csv`) exist they take precedence, keeping the pipeline faithful to the project scope in Chapter 3.


In [113]:
hive_logs_path = MAIN_DATA_DIR / "hive_logs_2008_2025.csv"
if hive_logs_path.exists():
    hive_df = pd.read_csv(hive_logs_path, parse_dates=["date"])
    print("Loaded hive logs from", hive_logs_path)
else:
    print("No local hive logs detected; relying on Würzburg telemetry for pretraining and climate-driven features.")
    hive_df = pd.DataFrame()



No local hive logs detected; relying on Würzburg telemetry for pretraining and climate-driven features.


In [114]:
merged = pd.merge(hive_df, weather_full, on="date", how="left") if not hive_df.empty else weather_full.copy()
merged["yield_proxy"] = merged.get("honey_yield_kg", pd.Series(0, index=merged.index)).fillna(0)
merged_path = MAIN_DATA_DIR / "merged_hive_weather_floral_2025.csv"
merged.to_csv(merged_path, index=False)
print("Merged hive/weather/floral ->", merged_path)
merged.head()


Merged hive/weather/floral -> /kaggle/working/content/main-data/merged_hive_weather_floral_2025.csv


,date,temp_max,temp_min,temp_mean,humidity_mean,rainfall_mm,wind_speed_max,cloud_cover_percent,ndvi_mean,yield_proxy
0,2008-01-01,24.9,16.4,20.3,74,1.2,15.1,53,0.6805,0
1,2008-01-02,25.8,14.1,20.2,71,0.8,14.3,19,NaN,0
2,2008-01-03,27.2,15.2,21.3,65,0.0,12.8,11,NaN,0
3,2008-01-04,27.6,15.4,22.2,63,0.1,12.2,26,NaN,0
4,2008-01-05,27.3,15.2,21.0,75,2.9,13.1,58,NaN,0


### Climate Feature Extraction

Derive clean hourly climate+NDVI features ready for inference.


In [115]:
climate_features = df_merged[['date','rainfall_mm','temp_mean','humidity_mean','ndvi_mean']].copy()
climate_features = climate_features.sort_values('date').set_index('date')
climate_features = climate_features.interpolate(limit_direction='both')
print('Climate feature frame:', climate_features.shape)


Climate feature frame: (6549, 4)


### Würzburg Tabular Inference

Apply pretrained HGB to climate features for yield probability curves.


In [129]:
WURZBURG_MODEL_PATH = MAIN_DATA_DIR / 'wurzb_hgb_yield.pkl'
if not WURZBURG_MODEL_PATH.exists():
    raise FileNotFoundError('Pretrained Würzburg HGB missing')
hgb_w = pd.read_pickle(WURZBURG_MODEL_PATH)
climate_X = climate_features[['rainfall_mm','temp_mean','humidity_mean','ndvi_mean']].fillna(0)
yield_probs = hgb_w.predict_proba(climate_X)[:,1]
climate_features['predicted_yield_prob'] = yield_probs
yield_forecast_path = MAIN_DATA_DIR / 'makueni_climate_yield_forecast.csv'
climate_features[['predicted_yield_prob']].to_csv(yield_forecast_path)
print('Saved climate yield forecast ->', yield_forecast_path)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- humidity_mean
- ndvi_mean
- rainfall_mm
- temp_mean
Feature names seen at fit time, yet now missing:
- flow
- humidity
- temperature
- weight
- weight_delta_24h


### Würzburg CNN Inference

Convert climate sequences to windows and run the pretrained CNN to estimate occupancy risk.


In [130]:
WINDOW = 24  # match Würzburg pretraining window
climate_sequences = []
for i in range(WINDOW, len(climate_features)):
    window = climate_features.iloc[i-WINDOW:i][['rainfall_mm','temp_mean','humidity_mean','ndvi_mean']].values
    if not np.any(np.isnan(window)):
        climate_sequences.append(window)
climate_sequences = np.array(climate_sequences, dtype=np.float32)
print('Climate sequences:', climate_sequences.shape)
cnn_model = HiveCNNBaseline(climate_sequences.shape[-1]).to(device)
pretrain_path = MAIN_DATA_DIR / 'wurzburg_sequence_pretrain.pt'
cnn_model.load_state_dict(torch.load(pretrain_path, map_location=device), strict=False)
cnn_model.eval()
with torch.no_grad():
    occupancy = torch.sigmoid(cnn_model(torch.tensor(climate_sequences).to(device))).cpu().numpy()
forecast = climate_features.iloc[WINDOW:].copy()
forecast['occupancy_risk'] = occupancy
climate_forecast_path = MAIN_DATA_DIR / 'makueni_climate_occupancy_forecast.csv'
forecast[['occupancy_risk']].to_csv(climate_forecast_path)
print('Saved occupancy forecast ->', climate_forecast_path)


Climate sequences: (6525, 24, 4)


NameError: name 'HiveCNNBaseline' is not defined

### Würzburg Dataset Staging

On Kaggle, stage the Würzburg datasets just like the acoustic source: attach a dataset named `wurzb-hive-telemetry` (or similar) under `/kaggle/input`, then copy it into `content/wurzburg/` so the pretraining block can load from both local runs and Kaggle sessions.


In [121]:
WURZBURG_INPUT_ROOT = Path('/kaggle/input')
WURZBURG_TARGET = CONTENT_ROOT / 'bee-hive-metrics'
if WURZBURG_INPUT_ROOT.exists():
    candidates = [p for p in WURZBURG_INPUT_ROOT.iterdir() if 'bee-hive-metrics' in p.name.lower() or 'bee-hive-metrics' in p.name.lower()]
    if candidates:
        source = candidates[0]
        WURZBURG_TARGET.mkdir(parents=True, exist_ok=True)
        for csv_path in source.glob('*.csv'):
            target_path = WURZBURG_TARGET / csv_path.name
            if not target_path.exists():
                shutil.copy(csv_path, target_path)
        print(f"Staged Würzburg telemetry from {source} -> {WURZBURG_TARGET}")
    else:
        print('No Würzburg dataset attached under /kaggle/input; using existing content/wurzburg if present.')
else:
    WURZBURG_TARGET.mkdir(parents=True, exist_ok=True)



Staged Würzburg telemetry from /kaggle/input/bee-hive-metrics -> /kaggle/working/content/bee-hive-metrics


### Würzburg/Schwartau Hive Telemetry Pretraining

To ground our synthetic Makueni logs in real telemetry, we ingest the Würzburg/Schwartau sensor datasets (`content/wurzburg/*`). These CSVs provide minute-level hive weight, entrance flow, and local temperature/humidity measurements from 2017–2019. We aggregate them into hourly/daily features, derive proxy yield/ stress labels via rolling weight deltas, and use them to pretrain the tabular and sequence models before adapting to Makueni's climate distribution.


In [122]:
from pathlib import Path
import pandas as pd

WURZBURG_DIR = CONTENT_ROOT / 'bee-hive-metrics'
if not WURZBURG_DIR.exists():
    raise FileNotFoundError(f"Missing Würzburg data at {WURZBURG_DIR}")

# Helper loader keeps timestamps sorted for consistent resampling
def load_sensor(name):
    path = WURZBURG_DIR / name
    df = pd.read_csv(path, parse_dates=['timestamp'])
    df = df.sort_values('timestamp').set_index('timestamp')
    return df

weight_df = load_sensor('weight_wurzburg.csv')
temp_df = load_sensor('temperature_wurzburg.csv')
humidity_df = load_sensor('humidity_wurzburg.csv')
flow_df = load_sensor('flow_wurzburg.csv')

print('Loaded sensors:', {
    'weight': weight_df.shape,
    'temperature': temp_df.shape,
    'humidity': humidity_df.shape,
    'flow': flow_df.shape
})

# Resample to hourly means and align
hourly = pd.DataFrame({
    'weight': weight_df['weight'].resample('1H').mean(),
    'temperature': temp_df['temperature'].resample('1H').mean(),
    'humidity': humidity_df['humidity'].resample('1H').mean(),
    'flow': flow_df['flow'].resample('1H').mean(),
})
hourly = hourly.interpolate(limit_direction='both')

# Derive proxy labels: positive weight change over 24h indicates nectar intake (yield), negative sustained drop indicates stress/harvest
hourly['weight_delta_24h'] = hourly['weight'].diff(24)
hourly['yield_positive'] = (hourly['weight_delta_24h'] > 0.5).astype(int)
hourly['stress_event'] = (hourly['weight_delta_24h'] < -1.0).astype(int)

hourly.reset_index(inplace=True)
print('Hourly feature set:', hourly.shape)
hourly.head()


Loaded sensors: {'weight': (1035861, 1), 'temperature': (958831, 1), 'humidity': (20845, 1), 'flow': (2071720, 1)}
Hourly feature set: (20865, 8)


,timestamp,weight,temperature,humidity,flow,weight_delta_24h,yield_positive,stress_event
0,2017-01-01 05:00:00,52.695098,-0.327590,92.406667,0.0,NaN,0,0
1,2017-01-01 06:00:00,52.685200,-0.409250,92.270000,0.0,NaN,0,0
2,2017-01-01 07:00:00,52.688667,-0.668364,92.575000,0.0,NaN,0,0
3,2017-01-01 08:00:00,52.674267,-0.966858,92.840000,0.0,NaN,0,0
4,2017-01-01 09:00:00,52.595320,-1.623189,93.640000,0.0,NaN,0,0


#### Würzburg Tabular Pretraining

We treat the hourly temperature/humidity/flow/weight statistics plus rolling deltas as features and pretrain a HistGradientBoostingClassifier to predict the proxy `yield_positive` label. This serves as an initialization for the Makueni model (via warm-start) and quantifies how real telemetry behaves before domain adaptation.


In [123]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

feature_cols_w = ['weight', 'temperature', 'humidity', 'flow', 'weight_delta_24h']
X_w = hourly[feature_cols_w].fillna(method='ffill').fillna(method='bfill')
y_w = hourly['yield_positive']

X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_w, y_w, test_size=0.2, random_state=42, stratify=y_w)

hgb_w = HistGradientBoostingClassifier(max_depth=6, learning_rate=0.08, max_iter=300, class_weight='balanced')
hgb_w.fit(X_train_w, y_train_w)

y_pred_w = hgb_w.predict(X_test_w)
y_prob_w = hgb_w.predict_proba(X_test_w)[:, 1]

print(classification_report(y_test_w, y_pred_w))
print('ROC-AUC:', roc_auc_score(y_test_w, y_prob_w))

# Persist for transfer learning
WURZBURG_MODEL_PATH = MAIN_DATA_DIR / 'wurzb_hgb_yield.pkl'
pd.to_pickle(hgb_w, WURZBURG_MODEL_PATH)
print('Saved Würzburg HGB model ->', WURZBURG_MODEL_PATH)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3940
           1       0.97      1.00      0.98       233

    accuracy                           1.00      4173
   macro avg       0.98      1.00      0.99      4173
weighted avg       1.00      1.00      1.00      4173

ROC-AUC: 0.9999771246813794
Saved Würzburg HGB model -> /kaggle/working/content/main-data/wurzb_hgb_yield.pkl


#### Transfer to Makueni Tabular Model

We reuse the Würzburg-trained gradient boosting model as initialization for the Makueni stress prediction: load `wurzb_hgb_yield.pkl`, continue training on the Makueni feature matrix, and compare against training-from-scratch baselines. This simple warm-start helps incorporate learned environmental response patterns despite limited local telemetry.


#### Würzburg Sequence Pretraining

We also slice the Würzburg hourly features into temporal windows to pretrain the PyTorch sequence backbone. This yields a model familiar with real hive dynamics before fine-tuning on Makueni's synthetic/log-based sequences.


In [ ]:
class HiveCNNBaseline(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.features(x)
        x = self.classifier(x)
        return x.squeeze(-1)

class HiveCNNRecurrent(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        self.layer_norm = nn.LayerNorm(128)
        self.gru = nn.GRU(128, 64, batch_first=True, bidirectional=True)
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.conv(x)
        x = x.transpose(1, 2)
        x = self.layer_norm(x)
        _, h_n = self.gru(x)
        h_n = h_n.transpose(0, 1).reshape(x.size(0), -1)
        x = self.classifier(h_n)
        return x.squeeze(-1)



In [127]:
# Slice Würzburg hourly features into fixed windows for sequence pretraining
WURZ_WINDOW = 24
sequence_cols_w = ['weight', 'temperature', 'humidity', 'flow', 'weight_delta_24h']
wurz_sequences = []
wurz_labels = []
for i in range(WURZ_WINDOW, len(hourly)):
    window = hourly.iloc[i-WURZ_WINDOW:i][sequence_cols_w].values
    label = hourly.iloc[i]['yield_positive']
    if not np.any(np.isnan(window)):
        wurz_sequences.append(window)
        wurz_labels.append(label)

wurz_sequences = np.array(wurz_sequences, dtype=np.float32)
wurz_labels = np.array(wurz_labels, dtype=np.float32)
print('Würzburg sequences:', wurz_sequences.shape)



Würzburg sequences: (20817, 24, 5)


##### Pretraining the Temporal CNN

We reuse the `SEQUENCE_MODEL_VARIANT='cnn'` architecture to pretrain on the Würzburg sequences, save the weights, and later load them as initialization for the Makueni sequence training.


In [128]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Lightweight dataset wrapper so PyTorch loaders can stream Würzburg sequences
class WurzburgDataset(Dataset):
    def __init__(self, sequences, labels):
        self.X = torch.tensor(sequences, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

X_train_wseq, X_val_wseq, y_train_wseq, y_val_wseq = train_test_split(
    wurz_sequences, wurz_labels, test_size=0.2, random_state=42, stratify=wurz_labels)

w_train_ds = WurzburgDataset(X_train_wseq, y_train_wseq)
w_val_ds = WurzburgDataset(X_val_wseq, y_val_wseq)
w_train_loader = DataLoader(w_train_ds, batch_size=128, shuffle=True)
w_val_loader = DataLoader(w_val_ds, batch_size=128, shuffle=False)

pretrain_model = HiveCNNBaseline(input_channels=len(sequence_cols_w)).to(device)
pretrain_optimizer = torch.optim.Adam(pretrain_model.parameters(), lr=1e-3, weight_decay=1e-4)
pretrain_criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor((len(y_train_wseq)-y_train_wseq.sum())/max(y_train_wseq.sum(),1), device=device))

def run_pretrain_epoch(loader, train=True):
    pretrain_model.train(train)
    total_loss=0
    preds=[]
    targets=[]
    for batch_X, batch_y in loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        pretrain_optimizer.zero_grad()
        with torch.set_grad_enabled(train):
            logits = pretrain_model(batch_X)
            loss = pretrain_criterion(logits, batch_y)
            if train:
                loss.backward()
                pretrain_optimizer.step()
        total_loss += loss.item()*batch_X.size(0)
        preds.append(torch.sigmoid(logits).detach().cpu().numpy())
        targets.append(batch_y.detach().cpu().numpy())
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    return total_loss/len(loader.dataset), roc_auc_score(targets, preds)

# Train for a few epochs and persist the best validation AUC weights
print('Pretraining sequence model on Würzburg data...')
best_auc = 0
for epoch in range(10):
    train_loss, train_auc = run_pretrain_epoch(w_train_loader, True)
    val_loss, val_auc = run_pretrain_epoch(w_val_loader, False)
    print(f'Epoch {epoch+1:02d}: train_loss={train_loss:.4f} AUC={train_auc:.3f} | val_loss={val_loss:.4f} AUC={val_auc:.3f}')
    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(pretrain_model.state_dict(), MAIN_DATA_DIR / 'wurzburg_sequence_pretrain.pt')
print('Saved Würzburg sequence weights ->', MAIN_DATA_DIR / 'wurzburg_sequence_pretrain.pt')



NameError: name 'HiveCNNBaseline' is not defined